In [1]:
import pandas as pd
import numpy as np
import ppscore as pps
import statsmodels as sm
import time
from sklearn.model_selection import KFold
from hpsklearn import HyperoptEstimator, any_classifier, any_preprocessing, random_forest, gaussian_nb, gradient_boosting, multinomial_nb
from hyperopt import tpe
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier

ModuleNotFoundError: No module named 'hpsklearn'

In [2]:
#Load data
pd.set_option('display.max_columns', None)
df_td = pd.read_csv('tic_2000_train_data.csv')
eval_df = pd.read_csv('tic_2000_eval_data.csv')#CARAVAN is renamed 'Target' in this set

#renaming the training data to match the test data.
df_td.rename(columns={'MOSTYPE': 'subtype_L0', 'MAANTHUI':'Num_houses', 'MGEMOMV' : 'Avg_hh_size',
                   'MGEMLEEF':'age_L1', 'MOSHOOFD': 'maintype_L2', 'MGODRK': 'romcath_L3',
                   'MGODPR': 'Protestant','MGODOV' : 'O_religion', 'MGODGE': 'N_religion','MRELGE' :'Married',
                   'MRELSA' : 'Living_together','MRELOV' : 'O_relation','MFALLEEN' : 'Singles','MFGEKIND' : 'hh_wo_child',
                   'MFWEKIND' : 'hh_w_child','MOPLHOOG' : 'H_lvl_edu','MOPLMIDD' : 'M_lvl_edu',
                   'MOPLLAAG' : 'L_lvl_edu','MBERHOOG' : 'H_status','MBERZELF' : 'Entrepreneur','MBERBOER' : 'Farmer',
                   'MBERMIDD' : 'Mid_management','MBERARBG' : 'Skld_labor','MBERARBO' : 'Unskld_labor',
                   'MSKA' : 'Soc_cls_A','MSKB1' : 'Soc_cls_B1','MSKB2' : 'Soc_cls_B2','MSKC' : 'Soc_cls_C',
                   'MSKD' : 'Soc_cls_D','MHHUUR' : 'R_house','MHKOOP' : 'O_house','MAUT1' : '1_car','MAUT2' : '2_cars',
                   'MAUT0' : 'N_car','MZFONDS' : 'Nat_Hlth_Serv','MZPART' : 'Prv_Hlth_Insur','MINKM30' : 'Inc_u_30k',
                   'MINK3045' : 'Inc_btw_30_45k','MINK4575' : 'Inc_btw_45_75k','MINK7512' : 'Inc_75_122k','MINK123M' : 'Inc_ovr_123k',
                   'MINKGEM' : 'Avg_inc','MKOOPKLA' : 'PP_cls','PWAPART' : 'Contri_prv_3p_insur','PWABEDR' : 'Firm_Contri_3p_ insur',
                   'PWALAND' : 'Ag_Contri_3p_insur','PPERSAUT' : 'Contri_car_pol','PBESAUT' : 'Contri_deliv_van_pol',
                   'PMOTSCO' : 'Contri_motorcycle/scooter_pol','PVRAAUT' : 'Contri_lorry_pol','PAANHANG' : 'Contri_trailer_pols',
                   'PTRACTOR' : 'Contri_tractor_pol','PWERKT' : 'Contri_ag_machine_pol','PBROM' : 'Contri_moped_pol',
                   'PLEVEN' : 'Contri_life_insur','PPERSONG' : 'Contri_prv_accid_insur_pol',
                   'PGEZONG' : 'Contri_fam_accid_insur_pol','PWAOREG' : 'Contri_disabl_insur_pol','PBRAND' : 'Contri_fire_pol',
                   'PZEILPL' : 'Contri_surfb_pol','PPLEZIER' : 'Contri_boat_pol','PFIETS' : 'Contri_bike_pol',
                   'PINBOED' : 'Contri_prop_insur_pol','PBYSTAND' : 'Contri_ss_insur_polo','AWAPART' : 'Num_prv_3p_insur',
                   'AWABEDR' : 'Num_firm_3p_insur','AWALAND' : 'Num_ag_3p_insur','APERSAUT' : 'Num_car_pol',
                   'ABESAUT' : 'Num_deliv_van_pol','AMOTSCO' : 'Num_motorcycle/scooter_pol', 'AVRAAUT' : 'Num_lorry_pol','AAANHANG': 'Num_trailer_pol',
                   'ATRACTOR' : 'Num_tractor_pol','AWERKT' : 'Num_ag_machines_pol','ABROM' : 'Num_moped_pol',
                   'ALEVEN' : 'Num_life_insur_pol', 'APERSONG' : 'Num_prv_accid_insur_pol','AGEZONG' : 'Num_fam_ccid_insur_pol',
                   'AWAOREG' : 'Num_disabl_insur_pol','ABRAND' :'Num_fire_pol','AZEILPL' :'Num_surfb_pol','APLEZIER' :'Num_boat_pol',
                   'AFIETS' :'Num_bike_pol','AINBOED' :'Num_prop_insur_pol','ABYSTAND' :'num_ss_insur_pol', 'CARAVAN' : 'Target'},
          inplace=True)
eval_df.rename(columns={'MOSTYPE': 'subtype_L0', 'MAANTHUI':'Num_houses', 'MGEMOMV' : 'Avg_hh_size',
                   'MGEMLEEF':'age_L1', 'MOSHOOFD': 'maintype_L2', 'MGODRK': 'romcath_L3',
                   'MGODPR': 'Protestant','MGODOV' : 'O_religion', 'MGODGE': 'N_religion','MRELGE' :'Married',
                   'MRELSA' : 'Living_together','MRELOV' : 'O_relation','MFALLEEN' : 'Singles','MFGEKIND' : 'hh_wo_child',
                   'MFWEKIND' : 'hh_w_child','MOPLHOOG' : 'H_lvl_edu','MOPLMIDD' : 'M_lvl_edu',
                   'MOPLLAAG' : 'L_lvl_edu','MBERHOOG' : 'H_status','MBERZELF' : 'Entrepreneur','MBERBOER' : 'Farmer',
                   'MBERMIDD' : 'Mid_management','MBERARBG' : 'Skld_labor','MBERARBO' : 'Unskld_labor',
                   'MSKA' : 'Soc_cls_A','MSKB1' : 'Soc_cls_B1','MSKB2' : 'Soc_cls_B2','MSKC' : 'Soc_cls_C',
                   'MSKD' : 'Soc_cls_D','MHHUUR' : 'R_house','MHKOOP' : 'O_house','MAUT1' : '1_car','MAUT2' : '2_cars',
                   'MAUT0' : 'N_car','MZFONDS' : 'Nat_Hlth_Serv','MZPART' : 'Prv_Hlth_Insur','MINKM30' : 'Inc_u_30k',
                   'MINK3045' : 'Inc_btw_30_45k','MINK4575' : 'Inc_btw_45_75k','MINK7512' : 'Inc_75_122k','MINK123M' : 'Inc_ovr_123k',
                   'MINKGEM' : 'Avg_inc','MKOOPKLA' : 'PP_cls','PWAPART' : 'Contri_prv_3p_insur','PWABEDR' : 'Firm_Contri_3p_ insur',
                   'PWALAND' : 'Ag_Contri_3p_insur','PPERSAUT' : 'Contri_car_pol','PBESAUT' : 'Contri_deliv_van_pol',
                   'PMOTSCO' : 'Contri_motorcycle/scooter_pol','PVRAAUT' : 'Contri_lorry_pol','PAANHANG' : 'Contri_trailer_pols',
                   'PTRACTOR' : 'Contri_tractor_pol','PWERKT' : 'Contri_ag_machine_pol','PBROM' : 'Contri_moped_pol',
                   'PLEVEN' : 'Contri_life_insur','PPERSONG' : 'Contri_prv_accid_insur_pol',
                   'PGEZONG' : 'Contri_fam_accid_insur_pol','PWAOREG' : 'Contri_disabl_insur_pol','PBRAND' : 'Contri_fire_pol',
                   'PZEILPL' : 'Contri_surfb_pol','PPLEZIER' : 'Contri_boat_pol','PFIETS' : 'Contri_bike_pol',
                   'PINBOED' : 'Contri_prop_insur_pol','PBYSTAND' : 'Contri_ss_insur_polo','AWAPART' : 'Num_prv_3p_insur',
                   'AWABEDR' : 'Num_firm_3p_insur','AWALAND' : 'Num_ag_3p_insur','APERSAUT' : 'Num_car_pol',
                   'ABESAUT' : 'Num_deliv_van_pol','AMOTSCO' : 'Num_motorcycle/scooter_pol', 'AVRAAUT' : 'Num_lorry_pol','AAANHANG': 'Num_trailer_pol',
                   'ATRACTOR' : 'Num_tractor_pol','AWERKT' : 'Num_ag_machines_pol','ABROM' : 'Num_moped_pol',
                   'ALEVEN' : 'Num_life_insur_pol', 'APERSONG' : 'Num_prv_accid_insur_pol','AGEZONG' : 'Num_fam_ccid_insur_pol',
                   'AWAOREG' : 'Num_disabl_insur_pol','ABRAND' :'Num_fire_pol','AZEILPL' :'Num_surfb_pol','APLEZIER' :'Num_boat_pol',
                   'AFIETS' :'Num_bike_pol','AINBOED' :'Num_prop_insur_pol','ABYSTAND' :'num_ss_insur_pol'},
          inplace=True)

df_list = [df_td, eval_df]
df = pd.concat(df_list)

df.describe()

Target feature in Training Data; Balance 
 0    5474
1     348
Name: CARAVAN, dtype: int64
Target feature in Target Data; Balance 
 0    3762
1     238
Name: Target, dtype: int64
Training Features Dataset; Shape (5822, 86)
Eval Features Dataset; Shape (4000, 85)
Target Feature Dataset; Shape (4000, 1)


In [4]:
#Creating dummy tables and leaving our Target variable out for the models.
pre_enc= df.drop('Target', axis=1)
post_enc_df = pd.get_dummies(df, prefix_sep="_", columns=pre_enc.columns)

new_df = post_enc_df[['Contri_boat_pol_0', 'Avg_inc_0', 'maintype_L2_10', 'Contri_fire_pol_2',
       'Contri_ss_insur_polo_0', 'maintype_L2_5', 'maintype_L2_4', 'R_house_2',
       'L_lvl_edu_7', 'H_lvl_edu_3', 'H_lvl_edu_0', 'L_lvl_edu_6',
       'maintype_L2_3', '1_car_4', '1_car_2', 'L_lvl_edu_9', 'Avg_inc_2',
       'Contri_fire_pol_1', 'Avg_inc_3', 'Contri_fire_pol_6', 'R_house_5',
       'PP_cls_3', 'PP_cls_5', 'Contri_prv_3p_insur_1', 'Num_fire_pol_2',
       'maintype_L2_1', 'maintype_L2_2', 'Contri_disabl_insur_pol_6',
       'H_lvl_edu_5', 'Num_fire_pol_1', '1_car_5', '1_car_6', 'PP_cls_4',
       'H_lvl_edu_4', 'H_lvl_edu_6', 'Contri_fire_pol_5', 'Avg_inc_5',
       'maintype_L2_8', 'R_house_1', '1_car_9', 'L_lvl_edu_8', 'R_house_6',
       'L_lvl_edu_2', 'maintype_L2_9', 'Contri_fire_pol_3', 'Avg_inc_4',
       'Avg_inc_7', 'L_lvl_edu_1', 'PP_cls_8', 'R_house_0', 'L_lvl_edu_0',
       '1_car_7', 'Contri_prv_3p_insur_2', 'H_lvl_edu_7',
       'Contri_ss_insur_polo_4', 'Contri_fire_pol_4', 'PP_cls_7',
       'Contri_car_pol_6', 'Contri_fam_accid_insur_pol_3','Target']]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = new_df.drop('Target', axis=1)
y = new_df.Target


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)


# Instantiate a HyperoptEstimator with the search space and number of evaluations

estim = HyperoptEstimator(classifier=gradient_boosting('my_clf'),
                          preprocessing=any_preprocessing('my_pre'),
                          algo=tpe.suggest,
                          max_evals=100,
                          trial_timeout=120)

# Search the hyperparameter space based on the data

estim.fit(X_train, y_train)

# Show the results

print(estim.score(X_test, y_test))
# 1.0

print( estim.best_model() )

In [ ]:
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, log_loss, confusion_matrix, plot_roc_curve, classification_report, balanced_accuracy_score, coverage_error
import matplotlib.pyplot as plt

classifier = GradientBoostingClassifier(learning_rate=0.030509817593472984,
                           loss='exponential', max_features=0.36684107529168053,
                           min_samples_leaf=8, n_estimators=287, presort='auto',
                           random_state=3)

classifier.fit(X_train,y_train)

predictions = classifier.predict(X_test)

print("accuracy score" '\n', accuracy_score(y_test, predictions))
print("model confusion matrix" '\n', confusion_matrix(y_test, predictions, normalize='all'))
print("classification_report" '\n', classification_report(y_test, predictions),'\n')
ax = plt.gca()
plt.rcParams['figure.figsize'] = (10, 10)
disp = plot_roc_curve(classifier, X_test, y_test, ax=ax, alpha=0.9)

In [ ]:
estim_rf = HyperoptEstimator(classifier=random_forest('my_clf'),
                          preprocessing=any_preprocessing('my_pre'),
                          algo=tpe.suggest,
                          max_evals=100,
                          trial_timeout=120)

# Search the hyperparameter space based on the data

estim_rf.fit(X_train, y_train)

# Show the results

print(estim_rf.score(X_test, y_test))
# 1.0

print( estim_rf.best_model() )

In [ ]:
estim_gbt = HyperoptEstimator(classifier=gradient_boosting('my_clf'),
                          preprocessing=any_preprocessing('my_pre'),
                          algo=tpe.suggest,
                          max_evals=100,
                          trial_timeout=120)

# Search the hyperparameter space based on the data

estim_gbt.fit(X_train, y_train)

# Show the results

print(estim_gbt.score(X_test, y_test))
# 1.0

print( estim_gbt.best_model() )

In [ ]:
estim_guas = HyperoptEstimator(classifier=gaussian_nb('my_clf'),
                          preprocessing=any_preprocessing('my_pre'),
                          algo=tpe.suggest,
                          max_evals=100,
                          trial_timeout=120)

# Search the hyperparameter space based on the data

estim_guas.fit(X_train, y_train)

# Show the results

print(estim_guas.score(X_test, y_test))
# 1.0

print( estim_guas.best_model() )

In [ ]:
estim_find = HyperoptEstimator( algo=tpe.suggest, 
                            max_evals=150, 
                            trial_timeout=60 )

estim_find.fit(X_train, y_train)

# Show the results

# 1.0

print( estim_find.best_model() )

In [ ]:
from hpsklearn import extra_trees

estim_et = HyperoptEstimator(classifier=extra_trees('my_clf'),
                          preprocessing=any_preprocessing('my_pre'),
                          algo=tpe.suggest,
                          max_evals=100,
                          trial_timeout=120)

# Search the hyperparameter space based on the data

estim_et.fit(X_train, y_train)

# Show the results

print(estim_et.score(X_test, y_test))
# 1.0

print( estim_et.best_model() )


In [ ]:
from hpsklearn import knn, decision_tree, ada_boost, linear_discriminant_analysis, quadratic_discriminant_analysis, xgboost_classification

estim_knn = HyperoptEstimator(classifier=knn('my_clf'),
                          preprocessing=any_preprocessing('my_pre'),
                          algo=tpe.suggest,
                          max_evals=100,
                          trial_timeout=120)

# Search the hyperparameter space based on the data

estim_knn.fit(X_train, y_train)

# Show the results

print(estim_knn.score(X_test, y_test))
# 1.0

print( estim_knn.best_model() )

In [ ]:
estim_dt = HyperoptEstimator(classifier=decision_tree('my_clf'),
                          preprocessing=any_preprocessing('my_pre'),
                          algo=tpe.suggest,
                          max_evals=100,
                          trial_timeout=120)

# Search the hyperparameter space based on the data

estim_dt.fit(X_train, y_train)

# Show the results

print(estim_dt.score(X_test, y_test))
# 1.0

print( estim_dt.best_model() )

In [ ]:
estim_adab = HyperoptEstimator(classifier=ada_boost('my_clf'),
                          preprocessing=any_preprocessing('my_pre'),
                          algo=tpe.suggest,
                          max_evals=100,
                          trial_timeout=120)

# Search the hyperparameter space based on the data

estim_adab.fit(X_train, y_train)

# Show the results

print(estim_adab.score(X_test, y_test))
# 1.0

print( estim_adab.best_model() )

In [ ]:
estim_lda = HyperoptEstimator(classifier=linear_discriminant_analysis('my_clf'),
                          preprocessing=any_preprocessing('my_pre'),
                          algo=tpe.suggest,
                          max_evals=100,
                          trial_timeout=120)

# Search the hyperparameter space based on the data

estim_lda.fit(X_train, y_train)

# Show the results

print(estim_lda.score(X_test, y_test))
# 1.0

print( estim_lda.best_model() )

In [ ]:
estim_qda = HyperoptEstimator(classifier=quadratic_discriminant_analysis('my_clf'),
                          preprocessing=any_preprocessing('my_pre'),
                          algo=tpe.suggest,
                          max_evals=100,
                          trial_timeout=120)

# Search the hyperparameter space based on the data

estim_qda.fit(X_train, y_train)

# Show the results

print(estim_qda.score(X_test, y_test))
# 1.0

print( estim_qda.best_model() )

In [ ]:
estim_xgb = HyperoptEstimator(classifier=xgboost_classification('my_clf'),
                          preprocessing=any_preprocessing('my_pre'),
                          algo=tpe.suggest,
                          max_evals=100,
                          trial_timeout=120)

# Search the hyperparameter space based on the data

estim_xgb.fit(X_train, y_train)

# Show the results

print(estim_xgb.score(X_test, y_test))
# 1.0

print( estim_xgb.best_model() )